In [2]:
import pandas as pd

game_df = pd.read_csv(r'C:\Users\ninuy\DSC 672\Data\NBA Team Box Score Stats Filtered.csv')


In [5]:
game_df.drop(columns=['Unnamed: 0'], inplace=True)


In [8]:
pd.set_option('display.max_columns', None)


In [17]:
filtered_df = game_df[[
    "Season", "Game_ID", "TEAM_NAME", "GAME_DATE", "MATCHUP", "WL",
    "HOME_MIN", "HOME_FGM", "HOME_FGA", "HOME_FG_PCT",
    "HOME_FG3M", "HOME_FG3A", "HOME_FG3_PCT",
    "HOME_FTM", "HOME_FTA", "HOME_FT_PCT",
    "HOME_OREB", "HOME_DREB", "HOME_REB", "HOME_AST", "HOME_STL", "HOME_BLK",
    "HOME_TOV", "HOME_PF", "HOME_PTS",
    "AWAY_MIN", "AWAY_FGM", "AWAY_FGA", "AWAY_FG_PCT",
    "AWAY_FG3M", "AWAY_FG3A", "AWAY_FG3_PCT",
    "AWAY_FTM", "AWAY_FTA", "AWAY_FT_PCT",
    "AWAY_OREB", "AWAY_DREB", "AWAY_REB", "AWAY_AST", "AWAY_STL", "AWAY_BLK",
    "AWAY_TOV", "AWAY_PF", "AWAY_PTS",
    "POINT_MARGIN"
]]

In [18]:
import pandas as pd

# 1. Convert GAME_DATE to datetime
filtered_df['GAME_DATE'] = pd.to_datetime(filtered_df['GAME_DATE'])

# 2. Define NBA seasons with start and end dates
seasons = [
    ("2017-18", "2017-10-01", "2018-06-30"),
    ("2018-19", "2018-10-01", "2019-06-30"),
    ("2019-20", "2019-10-01", "2020-10-11"),  # Bubble season
    ("2020-21", "2020-12-01", "2021-07-31"),
    ("2021-22", "2021-10-01", "2022-06-30"),
    ("2022-23", "2022-10-01", "2023-06-30"),
    ("2023-24", "2023-10-01", "2024-06-30"),
    ("2024-25", "2024-10-01", "2025-06-30")
]

season_ranges = [(name, pd.to_datetime(start), pd.to_datetime(end)) for name, start, end in seasons]

def get_season(date):
    for name, start, end in season_ranges:
        if start <= date <= end:
            return name
    return None

# 3. Create Season column
filtered_df['Season'] = filtered_df['GAME_DATE'].apply(get_season)

# 4. Sort DataFrame by TEAM_NAME, Season, GAME_DATE
filtered_df.sort_values(by=['TEAM_NAME', 'Season', 'GAME_DATE'], inplace=True)

# 5. Identify numeric columns starting with HOME_ or AWAY_ for rolling averages
rolling_cols = [
    col for col in filtered_df.columns 
    if col.startswith(('HOME_', 'AWAY_')) and filtered_df[col].dtype in ['float64', 'int64']
]

# 6. Compute lagged 5-game rolling averages grouped by TEAM_NAME and Season
for col in rolling_cols:
    filtered_df[f'{col}_5gm_avg'] = (
        filtered_df
        .groupby(['TEAM_NAME', 'Season'])[col]
        .transform(lambda x: x.shift(1).rolling(window=5, min_periods=5).mean())
    )


C:\Users\ninuy\AppData\Local\Temp\ipykernel_13908\1114193148.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['GAME_DATE'] = pd.to_datetime(filtered_df['GAME_DATE'])
C:\Users\ninuy\AppData\Local\Temp\ipykernel_13908\1114193148.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['Season'] = filtered_df['GAME_DATE'].apply(get_season)
C:\Users\ninuy\AppData\Local\Temp\ipykernel_13908\1114193148.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a D

C:\Users\ninuy\AppData\Local\Temp\ipykernel_13908\1114193148.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df[f'{col}_5gm_avg'] = (
C:\Users\ninuy\AppData\Local\Temp\ipykernel_13908\1114193148.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df[f'{col}_5gm_avg'] = (
C:\Users\ninuy\AppData\Local\Temp\ipykernel_13908\1114193148.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead


In [19]:
filtered_df

,Season,Game_ID,TEAM_NAME,GAME_DATE,MATCHUP,WL,HOME_MIN,HOME_FGM,HOME_FGA,HOME_FG_PCT,HOME_FG3M,HOME_FG3A,HOME_FG3_PCT,HOME_FTM,HOME_FTA,HOME_FT_PCT,HOME_OREB,HOME_DREB,HOME_REB,HOME_AST,HOME_STL,HOME_BLK,HOME_TOV,HOME_PF,HOME_PTS,AWAY_MIN,AWAY_FGM,AWAY_FGA,AWAY_FG_PCT,AWAY_FG3M,AWAY_FG3A,AWAY_FG3_PCT,AWAY_FTM,AWAY_FTA,AWAY_FT_PCT,AWAY_OREB,AWAY_DREB,AWAY_REB,AWAY_AST,AWAY_STL,AWAY_BLK,AWAY_TOV,AWAY_PF,AWAY_PTS,POINT_MARGIN,HOME_MIN_5gm_avg,HOME_FGM_5gm_avg,HOME_FGA_5gm_avg,HOME_FG_PCT_5gm_avg,HOME_FG3M_5gm_avg,HOME_FG3A_5gm_avg,HOME_FG3_PCT_5gm_avg,HOME_FTM_5gm_avg,HOME_FTA_5gm_avg,HOME_FT_PCT_5gm_avg,HOME_OREB_5gm_avg,HOME_DREB_5gm_avg,HOME_REB_5gm_avg,HOME_AST_5gm_avg,HOME_STL_5gm_avg,HOME_BLK_5gm_avg,HOME_TOV_5gm_avg,HOME_PF_5gm_avg,HOME_PTS_5gm_avg,AWAY_MIN_5gm_avg,AWAY_FGM_5gm_avg,AWAY_FGA_5gm_avg,AWAY_FG_PCT_5gm_avg,AWAY_FG3M_5gm_avg,AWAY_FG3A_5gm_avg,AWAY_FG3_PCT_5gm_avg,AWAY_FTM_5gm_avg,AWAY_FTA_5gm_avg,AWAY_FT_PCT_5gm_avg,AWAY_OREB_5gm_avg,AWAY_DREB_5gm_avg,AWAY_REB_5gm_avg,AWAY_AST_5gm_avg,AWAY_STL_5gm_avg,AWAY_BLK_5gm_avg,AWAY_TOV_5gm_avg,AWAY_PF_5gm_avg,AWAY_PTS_5gm_avg
0,2017-18,21700072,Atlanta Hawks,2017-10-27,ATL vs. DEN,L,240,37.0,89.0,0.416,14.0,33.0,0.424,12.0,12.0,1.000,11.0,32.0,43,23.0,7.0,4.0,12.0,21.0,100,240,42.0,91.0,0.462,4.0,20.0,0.200,17.0,23.0,0.739,11.0,35.0,46,25.0,8.0,4.0,11.0,12.0,105,-5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2017-18,21700085,Atlanta Hawks,2017-10-29,ATL vs. MIL,L,240,36.0,77.0,0.468,13.0,29.0,0.448,21.0,26.0,0.808,3.0,28.0,31,27.0,9.0,2.0,16.0,21.0,106,240,44.0,81.0,0.543,13.0,26.0,0.500,16.0,21.0,0.762,5.0,38.0,43,30.0,9.0,3.0,14.0,27.0,117,-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2017-18,21700123,Atlanta Hawks,2017-11-03,ATL vs. HOU,L,240,39.0,81.0,0.481,11.0,33.0,0.333,15.0,18.0,0.833,5.0,36.0,41,16.0,8.0,8.0,18.0,16.0,104,240,42.0,89.0,0.472,16.0,47.0,0.340,19.0,23.0,0.826,9.0,35.0,44,28.0,14.0,4.0,12.0,16.0,119,-15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2017-18,21700146,Atlanta Hawks,2017-11-06,ATL vs. BOS,L,240,39.0,81.0,0.481,13.0,26.0,0.500,16.0,20.0,0.800,5.0,31.0,36,24.0,8.0,4.0,14.0,22.0,107,240,39.0,81.0,0.481,14.0,36.0,0.389,18.0,22.0,0.818,7.0,35.0,42,28.0,3.0,3.0,13.0,21.0,110,-3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2017-18,21700204,Atlanta Hawks,2017-11-15,ATL vs. SAC,W,240,50.0,79.0,0.633,16.0,32.0,0.500,10.0,16.0,0.625,9.0,44.0,53,40.0,9.0,6.0,18.0,23.0,126,240,31.0,88.0,0.352,6.0,28.0,0.214,12.0,20.0,0.600,11.0,18.0,29,18.0,12.0,2.0,12.0,19.0,80,46,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9514,2024-25,22401075,Washington Wizards,2025-03-29,WAS vs. BKN,L,240,37.0,85.0,0.435,9.0,34.0,0.265,29.0,33.0,0.879,7.0,33.0,40,21.0,5.0,7.0,6.0,15.0,112,240,39.0,88.0,0.443,18.0,46.0,0.391,19.0,20.0,0.950,12.0,37.0,49,25.0,2.0,4.0,12.0,23.0,115,-3,240.0,39.0,87.4,0.4468,13.6,37.8,0.3590,21.2,28.2,0.7568,9.2,32.0,41.2,24.6,9.2,7.0,14.8,17.6,112.8,240.0,47.6,98.2,0.4874,16.0,40.4,0.3862,17.8,22.8,0.7906,15.4,38.4,53.8,29.8,8.8,4.6,13.0,21.2,129.0
9515,2024-25,22401092,Washington Wizards,2025-03-31,WAS vs. MIA,L,240,34.0,86.0,0.395,13.0,43.0,0.302,13.0,18.0,0.722,5.0,34.0,39,22.0

In [20]:
filtered_df.to_csv(r'C:\Users\ninuy\DSC 672\Data\NBA Team Box Score Stats Filtered with Rolling.csv', index=False)
